<a href="https://colab.research.google.com/github/dakilaledesma/NCBG/blob/main/test/MPQ_Annotations_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading of datasets and prereqs

In [ ]:
%%bash
# Test subset of full dataset
curl -o sept13_testset.zip https://herbarium.ucht.cloud/s/dityq9AyGJNXY4T/download
unzip -q sept13_testset
mv /content/content/images /content/images
rm -rf /content/content

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4091M  100 4091M    0     0  70.8M      0  0:00:57  0:00:57 --:--:-- 73.3M


In [ ]:
%%bash
# Multilabel parts model ver 2b
curl -o mp_2b.zip https://herbarium.ucht.cloud/s/w88BAyFkjxkyFDd/download
unzip -q mp_2b.zip
mv /content/content/drive/MyDrive/UNC/NCBG/mp_2b /content/mp_2b
rm -rf /content/content

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2265M  100 2265M    0     0  73.3M      0  0:00:30  0:00:30 --:--:-- 86.2M


In [ ]:
%%bash
# Quality model
curl -o quality_model.zip https://herbarium.ucht.cloud/s/X25JoqKjP9Hxaxw/download
unzip -q quality_model.zip
mv /content/content/drive/MyDrive/UNC/NCBG/output/20220904-184244-convnext_large_384_in22ft1k-600 /content/quality_model
rm -rf /content/content

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2246M  100 2246M    0     0  60.2M      0  0:00:37  0:00:37 --:--:-- 81.1M


In [ ]:
%%bash
# Prerequisites
pip install -U tensorflow-gpu
apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The followin

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires keras<2.9,>=2.8.0rc0, but you have keras 2.10.0 which is incompatible.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.0 which is incompatible.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorflow-estimator<2.9,>=2.8, but you have tensorflow-estimator 2.10.0 which is incompatible.




In [ ]:
! git clone https://github.com/rwightman/pytorch-image-models.git

Cloning into 'pytorch-image-models'...
remote: Enumerating objects: 11108, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 11108 (delta 12), reused 15 (delta 5), pack-reused 11076
Receiving objects: 100% (11108/11108), 20.55 MiB | 25.94 MiB/s, done.
Resolving deltas: 100% (8157/8157), done.


# Model predictions

In [ ]:
# Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.applications.convnext import ConvNeXtLarge
from tensorflow.keras.layers import Dense
import tensorflow_addons as tfa
import keras
from glob import glob
import os
from PIL import Image, UnidentifiedImageError
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

/usr/local/lib/python3.7/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.7.0 and strictly below 2.10.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [ ]:
# Predefined class dict
class_dict = {'aspect': 0, 'bud': 1, 'fall-color': 2, 'flower': 3, 'fruit': 4, 'leaf': 5, 'leaf-underside': 6, 'other': 7, 'root': 8, 'seed': 9, 'side-by-side': 10, 'stem': 11, 'venation': 12, 'vestiture': 13, 'whole-plant': 14}
class_dict = {v: k for k, v in class_dict.items()}

In [ ]:
# Dictionary to store all of the predictions
pred_array = defaultdict(dict)

Quality model

In [ ]:
# Quality model predictions (this results in a topk_ids.csv file)
! python -u ./pytorch-image-models/inference.py images -b 32 --input-size 3 600 600 --num-classes 3 --no-test-pool --checkpoint /content/quality_model/last.pth.tar --model convnext_large_384_in22ft1k

Loaded state_dict from checkpoint '/content/quality_model/last.pth.tar'
Model convnext_large_384_in22ft1k created, param count: 196234947
Predict: [0/2959] Time 9.843 (9.843)
Predict: [10/2959] Time 1.369 (2.133)
Predict: [20/2959] Time 1.391 (1.770)
Predict: [30/2959] Time 1.370 (1.647)
Predict: [40/2959] Time 1.371 (1.580)
Predict: [50/2959] Time 1.371 (1.540)
Predict: [60/2959] Time 1.379 (1.513)
Predict: [70/2959] Time 1.378 (1.494)
Predict: [80/2959] Time 1.375 (1.479)
Predict: [90/2959] Time 1.380 (1.468)
Predict: [100/2959] Time 1.372 (1.459)
Predict: [110/2959] Time 1.374 (1.451)
Predict: [120/2959] Time 1.374 (1.445)
Predict: [130/2959] Time 1.374 (1.440)
Predict: [140/2959] Time 1.377 (1.436)
Predict: [150/2959] Time 1.376 (1.432)
Predict: [160/2959] Time 1.391 (1.430)
Predict: [170/2959] Time 1.374 (1.427)
Predict: [180/2959] Time 1.378 (1.424)
Predict: [190/2959] Time 1.375 (1.422)
Predict: [200/2959] Time 1.377 (1.419)
Predict: [210/2959] Time 1.370 (1.417)
Predict: [220/2

In [ ]:
# Gather predictions from output file
qpred_file = open("topk_ids.csv")
qpred_lines = qpred_file.readlines()

qual_dict = {}
for l in qpred_lines:
  l = l.strip().split(",")
  bn, top = l[0], l[1]
  qual_dict[bn] = top

Parts model


In [ ]:
# Load the model
opt = tfa.optimizers.AdaBelief(learning_rate=1e-3)
custom_objects = {"AdaBelief": opt}

with keras.utils.custom_object_scope(custom_objects):
  model = load_model("mp_2b")

In [ ]:
images_glob = list(glob("images/*.*"))
batched_images = np.array(images_glob)
batched_images = np.array_split(batched_images, len(images_glob) / 32)

In [ ]:
# Create predictions for all test set images
for batch in tqdm(batched_images):
  pred_images = []
  proc_batch_ims = []
  for im_fn in batch: # Inner for-loop to process images individually with their basenames
    bn = os.path.basename(im_fn)
    pred_quality = qual_dict[bn]

    bn = f"{bn.replace('.jpg', '')}.jpg" # Had a filename extension mishap
    idx, rest = bn.split("__")
    bn_split = rest.split("_")
    directory, orig_bn = [bn_split[0], '_'.join(bn_split[1:])]

    pred_array[orig_bn]["directory"] = directory
    if pred_quality is not None:
      pred_array[orig_bn]["imagequality"] = pred_quality

    try:
      im_o = Image.open(im_fn)
      im = im_o.resize((448, 448))
      if im.mode == 'L':
        im = im.convert('RGB')
      im = np.array(im, dtype=int)
      im = np.reshape(im, (448, 448, 3))
    except:
      continue

    pred_images.append(im)
    proc_batch_ims.append(orig_bn)

  pred = np.array(model(np.array(pred_images))) # Model predictions operate on the outer for-loop per batch
  for orig_bn, p in zip(proc_batch_ims, pred): # Deconstruct and reassign predictions to each basename
    p = [round(v, 4) for v in p]
    pred_labels = ';'.join([class_dict[idx] for idx, v in enumerate(p) if v > 0.1])
    pred_array[orig_bn]["image_context"] = pred_labels
  

  0%|          | 0/2959 [00:00<?, ?it/s]

In [ ]:
print(list(pred_array.items())[:50])

[('rtw_29116.jpg', {'directory': 'rtw', 'imagequality': '2', 'image_context': 'flower'}), ('ed_20547.jpg', {'directory': 'ed', 'imagequality': '1', 'image_context': 'aspect;leaf'}), ('ed_20278.jpg', {'directory': 'ed', 'imagequality': '1', 'image_context': 'leaf;whole-plant'}), ('rtw_19134.jpg', {'directory': 'rtw', 'imagequality': '1', 'image_context': 'flower'}), ('rtw_01946.jpg', {'directory': 'rtw', 'imagequality': '2', 'image_context': 'flower'}), ('ed_07217.jpg', {'directory': 'ed', 'imagequality': '2', 'image_context': 'whole-plant'}), ('ed_17500.jpg', {'directory': 'ed', 'imagequality': '2', 'image_context': 'aspect'}), ('rtw_10053.jpg', {'directory': 'rtw', 'imagequality': '2', 'image_context': 'leaf'}), ('rtw_30448.jpg', {'directory': 'rtw', 'imagequality': '2', 'image_context': 'flower'}), ('rtw_16920.jpg', {'directory': 'rtw', 'imagequality': '2', 'image_context': 'flower'}), ('bas_04798.jpg', {'directory': 'bas', 'imagequality': '2', 'image_context': 'aspect'}), ('gpf_0481

In [ ]:
out_list = ["file,directory,imagequality,image_context"]
for im_bn, metadata in pred_array.items():
  try:
    out_list.append(f"{im_bn},{metadata['directory']},{metadata['imagequality']},{metadata['image_context']}")
  except KeyError:
    pass

In [ ]:
out_file = open("pred_list.csv", 'w')
out_file.write('\n'.join(out_list))
out_file.close()

In [ ]:
! cp pred_list.csv drive/MyDrive/UNC/NCBG/pred_list.csv